In [153]:
import pandas as pd
import numpy as np

In [154]:
# Datos de entrenamiento
data = np.array([
            ['','Pink Floyd','The Beatles','R.E.M.','Nirvana','Queen','Oasis','Class'],
            ['P1', 1, 0, 0, 1, 1, 1, 'J'],
            ['P2', 1, 1, 0, 1, 1, 0, 'J'],
            ['P3', 1, 1, 1, 0, 0, 1, 'J'],
            ['P4', 1, 0, 1, 0, 0, 1, 'J'],
            ['P5', 1, 0, 0, 0, 1, 0, 'J'],
            ['P6', 1, 1, 1, 0, 0, 0, 'J'],
            ['P7', 1, 1, 0, 0, 1, 1, 'A'],
            ['P8', 1, 1, 1, 0, 0, 1, 'A'],
            ['P9', 1, 1, 1, 1, 1, 0, 'A'],
            ['P10',1, 1, 1, 0, 1, 0, 'A'],
            ['P11',1, 1, 1, 0, 1, 1, 'A'],
            ['P12',1, 1, 0, 1, 1, 0, 'A'],
            ['P13',1, 1, 0, 1, 0, 0, 'A']])
    
dF=pd.DataFrame(data=data[1:,1:],
                  index=data[1:,0],
                  columns=data[0,1:])
dF
dF.index

Index(['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11',
       'P12', 'P13'],
      dtype='object')

In [172]:
# Cálculo de P(band|clase) 
def prob_of_band_given_class(band,cl):
    Nc=0
    nc_band=0
    for k in dF.index:
        if dF.loc[k]['Class'] == cl:
            Nc += 1
            if dF.loc[k][band] == '1':
                nc_band += 1
    return nc_band/Nc

# Cálculo de P(clase)
def prob_of_class(cl):
    counter=0
    for k in dF.index:
        if dF.loc[k]['Class'] == cl:
            counter +=1
    return counter/len(dF.index)

print(len(dF.index))
prob_of_class('A')

# Cálculo de P(clase|persona) con MVE, la probabilidad de que una persona dada pertenezca a la clase
def mve_prob_of_being_of_class(cl,person):
    p=0
    p= prob_of_class(cl)
    for k in dF.columns[:-1]:
        if person[k] == '1':
            p *= prob_of_band_given_class(k,cl)
        else:
            p *= (1-prob_of_band_given_class(k,cl))
    return p


13


In [173]:
# Clasificador bayesiano ingenuo MEV. Clasifica un individuo como 'A' (adulto) o 'J' (joven)
def map_nb_classifier1(person):
    if mve_prob_of_being_of_class('A',person) < mve_prob_of_being_of_class('J',person):
        return 'J'
    if map_prob_of_being_of_class('J',person) < mve_prob_of_being_of_class('A',person):
        return 'A'
    #else:
     #   if np.random(0,1) < 0.5:
      #      return 'A'
       # else:
        #    return 'J'
        
for k in dF.index:
    print(k,mve_nb_classifier1(dF.loc[k]),dF.loc[k]['Class'])
        
# Testeo sobre conjunto de entrenamiento
#for l in dF.index:
#    for k in dF.columns:
#        personl = {k:dF.loc[l][k] for k in dF.columns}
#    print(l,' ',mve_nb_classifier1(personl),dF.loc[l]['Class'])

P1 J J
P2 A J
P3 A J
P4 J J
P5 J J
P6 A J
P7 A A
P8 A A
P9 A A
P10 A A
P11 A A
P12 A A
P13 A A


In [174]:
# Datos para predicciones
tdata = np.array([    
        ['','Pink Floyd','The Beatles', 'R.E.M.','Nirvana','Queen','Oasis'],
        ['x1',1, 1, 0, 1, 1, 0],
        ['x2',1, 0, 1, 1, 1, 1],
        ['x3',1, 1, 0, 0, 0, 0],
        ['x4',1, 1, 1, 1, 1, 1],
        ['x5',0, 1, 1, 1, 1, 1]])

testdF=pd.DataFrame(data=tdata[1:,1:],
                  index=tdata[1:,0],
                  columns=tdata[0,1:])
testdF

#for l in testdF.index:
 #   testl = {k:testdF.loc[l][k] for k in testdF.columns[0:]}
  #  print(l,testl,mve_nb_classifier1(testl))

#print(prob_of_being_of_class('A',testdF.loc['x4']),
#prob_of_being_of_class('J',testdF.loc['x4']))

,Pink Floyd,The Beatles,R.E.M.,Nirvana,Queen,Oasis
x1,1,1,0,1,1,0
x2,1,0,1,1,1,1
x3,1,1,0,0,0,0
x4,1,1,1,1,1,1
x5,0,1,1,1,1,1


In [175]:
# Predicciones
for k in testdF.index:
    print(mve_nb_classifier1(testdF.loc[k]))

A
J
A
A
None


In [177]:
# Suponemos que P(C) tiene distribución Bernoulli B(q). Buscamos q.

def qmap(cl)
    Nc=0
    nc_band=0
    for k in dF.index:
        if dF.loc[k]['Class'] == cl:
            Nc += 1
            if dF.loc[k][band] == '1':
                nc_band += 1
    p_att = (nc_band+2-1)/(Nc+6-2)
    p_class = (Nc+2-1)/(len(dF.index)+Nc+2-2)

# Cálculo de P(clase|persona) bajo MAP, la probabilidad de que una persona dada pertenezca a la clase
def map_prob_of_being_of_class(cl,person):
    
    
    p = prob_of_class(cl)
    for k in dF.columns[:-1]:
        if person[k] == '1':
            p *= prob_of_band_given_class(k,cl)
        else:
            p *= (1-prob_of_band_given_class(k,cl))
    return p
